in Batch GD : evry update uses : **ALL n training samples**
Imagine:
- 10 million samples
- 100 features
 >  Every single update must scan all 10 million rows.

**That’s extremely slow.**

## Problems with Batch Gradient Descent

### 1. High Computational Cost

Batch Gradient Descent uses all training samples to compute the gradient:

$$
\nabla J(\theta) = \frac{1}{n} \sum_{i=1}^{n} \nabla L_i(\theta)
$$

If the dataset is large (e.g., millions of samples), each update becomes computationally expensive.

Time complexity per update:

$$
O(n \cdot d)
$$

Where:
- $n$ = number of samples  
- $d$ = number of features  

---

### 2. Slow Parameter Updates

Batch GD performs only one update per epoch.

If the dataset is large:
- One epoch itself is costly.
- Learning becomes slow.

The model waits to process the entire dataset before making a single update.

---

### 3. High Memory Requirement

Batch GD assumes the entire dataset fits in memory.

In real-world applications:
- Data may be too large to fit into RAM.
- Data may arrive in streaming form.

Batch GD becomes impractical in such cases.

---

### 4. Poor Scalability

For very large datasets:
- Computing full gradients repeatedly is inefficient.
- Training time increases significantly.

Batch GD does not scale well to big data problems.

---

### 5. Deterministic Behavior

Batch GD always follows the exact gradient direction.

While this makes convergence smooth, it can:
- Get stuck at saddle points
- Be less effective in non-convex optimization problems

---

>In SGD, parameters are updated after processing each individual training sample.

# SGD

## Advantages of Stochastic Gradient Descent (SGD)

### 1. Faster Updates

SGD updates parameters after processing a single training sample:

$$
\theta = \theta - \eta \nabla L_i(\theta)
$$

This allows the model to start learning immediately instead of waiting for the entire dataset.

---

### 2. Scales to Large Datasets

Since it processes one sample at a time:
- It does not require loading the full dataset into memory.
- It works well for very large datasets.

---

### 3. Lower Memory Usage

SGD only needs one sample in memory for each update.  
This makes it suitable for streaming data and online learning.

---

### 4. Can Escape Saddle Points

Because updates are noisy, SGD does not strictly follow the exact gradient direction.  
This noise can help escape saddle points in non-convex optimization problems.

---

### 5. Faster Initial Convergence

SGD often makes rapid progress during early training stages.

---

## Disadvantages of Stochastic Gradient Descent (SGD)

### 1. Noisy Updates

Since gradients are computed from a single sample, updates fluctuate heavily.  
The loss curve appears zig-zag rather than smooth.

---

### 2. Unstable Convergence

SGD may oscillate around the minimum instead of converging smoothly.

---

### 3. Requires Careful Learning Rate Tuning

A learning rate that is too large may cause divergence.  
A learning rate that is too small may result in slow convergence.

---

### 4. May Not Reach Exact Minimum

Because of noisy updates, SGD may settle near the minimum rather than exactly at it.

---

## 

SGD is:
- Fast
- Memory efficient
- Scalable

But:
- Noisy
- Less stable than Batch Gradient Descent

This is why Mini-Batch Gradient Descent is commonly used in practice.


__________

## code 

In [3]:
import numpy as np
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

from sklearn.linear_model import LinearRegression
import time

In [4]:
X,y = load_diabetes(return_X_y = True)

In [5]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=32)

In [6]:
reg = LinearRegression()
reg.fit(X_train,y_train)

,fit_intercept,True
,copy_X,True
,tol,1e-06
,n_jobs,None
,positive,False


In [7]:
print(reg.coef_)
print(reg.intercept_)

[  32.50000078 -274.2083569   501.52744069  368.7931961  -822.31120988
  504.23867319  101.57697155  158.16545687  744.91541712   78.40553632]
154.31230345901605


In [8]:
y_pred = reg.predict(X_test)

In [9]:
r2_score(y_test , y_pred)

0.4522338796354569

In [10]:
X_train.shape

(353, 10)

## making own SGD class

## Formulation

### Model

For a single training example $(x_i, y_i)$:

$$
\hat{y}_i = \mathbf{w}^T x_i + b
$$

Where:

- $\mathbf{w} \in \mathbb{R}^d$ (weight vector)
- $b \in \mathbb{R}$ (bias term)

---

### Loss Function (Per Sample)

Using squared error loss:

$$
L_i(\mathbf{w}, b) = (y_i - \hat{y}_i)^2
$$

---

### Error Term

$$
e_i = y_i - \hat{y}_i
$$

---

### Gradients (Per Sample)

Gradient with respect to weights:

$$
\frac{\partial L_i}{\partial \mathbf{w}} =
-2 x_i (y_i - \hat{y}_i)
$$

Gradient with respect to bias:

$$
\frac{\partial L_i}{\partial b} =
-2 (y_i - \hat{y}_i)
$$

---

### SGD Update Rules

After computing gradients for a single sample, parameters are updated immediately.

Weight update:

$$
\mathbf{w} =
\mathbf{w}
-
\eta
\left(
-2 x_i (y_i - \hat{y}_i)
\right)
$$

Bias update:

$$
b =
b
-
\eta
\left(
-2 (y_i - \hat{y}_i)
\right)
$$

---

### Simplified Update Form

Since minus times minus becomes plus:

$$
\mathbf{w} =
\mathbf{w}
+
2 \eta x_i (y_i - \hat{y}_i)
$$

$$
b =
b
+
2 \eta (y_i - \hat{y}_i)
$$

---

### Key Difference from Batch Gradient Descent

Batch Gradient Descent:

$$
\frac{\partial J}{\partial \mathbf{w}} =
-\frac{2}{n}
\sum_{i=1}^{n}
x_i (y_i - \hat{y}_i)
$$

Stochastic Gradient Descent:

$$
\frac{\partial J}{\partial \mathbf{w}} \approx
-2 x_i (y_i - \hat{y}_i)
$$

There is:
- No summation over all samples
- No division by $n$
- One update per sample


In [11]:
class SGDRegressor:
    
    def __init__(self, learning_rate=0.01, epochs=100):
        self.coef_ = None
        self.intercept_ = None
        self.lr = learning_rate
        self.epochs = epochs
        
    def fit(self, X_train, y_train):
        
        n_samples, n_features = X_train.shape
        
        self.intercept_ = 0
        self.coef_ = np.zeros(n_features)
        
        for epoch in range(self.epochs):
            
            indices = np.random.permutation(n_samples)
            
            for idx in indices:
                
                y_hat = np.dot(X_train[idx], self.coef_) + self.intercept_
                
                error = y_train[idx] - y_hat
                
                d_intercept = -2 * error # as we are using one row 
                d_coef = -2 * error * X_train[idx]
                
                self.intercept_ -= self.lr * d_intercept
                self.coef_ -= self.lr * d_coef
        
        return self
    
    def predict(self, X_test):
        return np.dot(X_test, self.coef_) + self.intercept_


In [17]:
sgd = SGDRegressor(epochs=100)

In [18]:
start = time.time()
sgd.fit(X_train , y_train )
print("Time taken in SGD : " , time.time() - start)

Time taken in SGD :  0.33869075775146484


In [19]:
print(sgd.coef_)
print(sgd.intercept_)

[  45.45158681 -190.60249566  442.63822422  320.78531285  -30.98142631
  -86.50307464 -203.57104366  121.41777153  372.11805838  149.75810407]
146.57015663473646


In [20]:
y_pred1 = sgd.predict(X_test)

In [21]:
r2_score(y_test , y_pred)

0.4522338796354569

#### Time comparison 

#### Comparison of Batch Gradient Descent and Stochastic Gradient Descent

Assume:

- Number of epochs = $e$
- Number of samples = $n$
- Number of features = $d$

---

## What Does One Epoch Mean?

An epoch means one full pass over the entire dataset.

- In **Batch Gradient Descent (BGD)**:
  - One update per epoch
  - Uses all $n$ samples to compute gradient

- In **Stochastic Gradient Descent (SGD)**:
  - $n$ updates per epoch
  - Uses one sample per update

---

## Computational Cost Per Epoch

### Batch Gradient Descent

Per epoch:
- Compute predictions for all $n$ samples
- Compute full gradient
- Perform 1 update

Time complexity per epoch:

$$
O(n \cdot d)
$$

---

### Stochastic Gradient Descent

Per epoch:
- For each of the $n$ samples:
  - Compute prediction
  - Compute gradient
  - Update parameters

Total time complexity per epoch:

$$
O(n \cdot d)
$$

---

## Number of Parameter Updates

Over $e$ epochs:

- Batch GD performs:
  
  $$
  e \text{ updates}
  $$

- SGD performs:

  $$
  e \times n \text{ updates}
  $$

SGD performs significantly more parameter updates.

---

## Which Is Faster?

It depends on what "faster" means.

### Faster Initial Learning

SGD often reduces the loss quickly at the beginning because:
- It updates parameters immediately.
- It does not wait for the entire dataset.

---

### Faster Stable Convergence

Batch GD:
- Produces smooth updates.
- Converges more steadily.
- Often requires fewer epochs for stable convergence.

---

## Practical Insight

- For small datasets: Batch GD is simple and stable.
- For very large datasets: SGD is more practical and scalable.
- In modern machine learning, Mini-Batch Gradient Descent is commonly used as a compromise between the two.


# but why here batch GD is faster ?

#### 1. Python Loop Overhead

In the Batch Gradient Descent implementation, gradient computation is fully vectorized:

$$
\nabla J = -\frac{2}{n} X^T (y - \hat{y})
$$

This uses optimized NumPy operations written in C, which are very fast.

In contrast, the SGD implementation uses Python loops:

```python
for idx in indices:

#### 2. Number of Parameter Updates

Assume:

- Number of samples = $n$
- Number of epochs = $e$

Batch Gradient Descent performs:

$$
e \text{ updates}
$$

Stochastic Gradient Descent performs:

$$
e \times n \text{ updates}
$$

SGD performs many more parameter updates, which increases total computation time.


#### 3. Small Dataset Effect

For small datasets:

- The entire dataset easily fits in memory.
- Matrix operations are highly efficient.
- Vectorized NumPy operations are extremely fast.

In such cases, Batch Gradient Descent is often faster than SGD.

---

#### 4. Vectorization Advantage

Batch Gradient Descent uses large matrix multiplications:

$$
\nabla J = -\frac{2}{n} X^T (y - \hat{y})
$$

These operations are optimized in low-level C code inside NumPy, making them very efficient.

In contrast, SGD performs many small updates inside Python loops, which reduces computational efficiency.

---

#### 5. When Does SGD Become Advantageous?

SGD becomes practical and advantageous when:

- The dataset is extremely large.
- The dataset does not fit into memory.
- Data arrives in streaming or online form.
- Full gradient computation becomes computationally expensive.

In large-scale settings, SGD scales better despite appearing slower in small Python implementations.


# when to use stochastic GD

1) when you have big data
2) non convex function

##### Problem in SGD

- solution is fluctuated (has noises)

## learning schedules

```python
t0,t1 = 5,50
def learning_rate(t):
    return t0/(t+t1)
for i in range(epochs):
   for j in range(X.shape[0]):
     lr = learning_rate(i * X.shape[0] + j)

## Sklearn SGDRegressor class

In [1]:
from sklearn.linear_model import SGDRegressor

In [2]:
from sklearn.linear_model import SGDRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_diabetes

# Load data
X, y = load_diabetes(return_X_y=True)

# Scale features (VERY IMPORTANT for SGD)
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Create model
sgd = SGDRegressor(max_iter=1000, eta0=0.01, random_state=42)

# Fit model
sgd.fit(X_train, y_train)

# Predict
y_pred = sgd.predict(X_test)

In [3]:
print(sgd.coef_)
print(sgd.intercept_)

[  2.01012851 -11.3683114   26.51176118  16.47680046  -6.0447379
  -4.66170233 -10.28473304   6.52887228  20.6797854    2.64643462]
[151.41172729]


In [5]:
from sklearn.metrics import r2_score
r2_score(y_test , y_pred)

0.4555738341969293